In [1]:
import cv2
import numpy as np
import sys

## webcam 띄우기

In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
while True :
    ret, frame = cap.read()
    
    if not ret :
        print('frame read failed')
        break
    
    frame = cv2.flip(frame, 1)

    cv2.imshow('Result', frame)
    
    key = cv2.waitKey(20)
    if key == 27 :
        break

cap.release()
cv2.destroyAllWindows()

## BGR -> HSV 색상 변환 및 객체 mask 생성

In [ ]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
while True :
    ret, frame = cap.read()
    
    if not ret :
        print('frame read failed')
        break
    
    frame = cv2.flip(frame, 1)
    
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    img_mask = cv2.inRange(img_hsv, (100, 200, 0), (140, 255, 255))
    
    # 모폴로지 연산 함수를 이용한 닫힘 연산(팽창 -> 침식)으로 틈 채우기
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11,11))
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_CLOSE, kernel, iterations = 3)

    cv2.imshow('Blue', img_mask)
    cv2.imshow('Result', frame)
    
    key = cv2.waitKey(20)
    if key == 27 :
        break

cap.release()
cv2.destroyAllWindows()

## 객체 중심점 tracking을 통해 webcam 영상에 선 그리기

In [ ]:
def draw_ball_location(frame, locations) :
    for i in range(len(locations)-1) :
        if locations[0] is None or locations[1] is None :
            continue
        cv2.line(frame, tuple(locations[i]), tuple(locations[i+1]), (0, 255, 255), 7)
    return frame

img = np.ones((500,700,3), np.uint8)*255
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
list_ball_location = []
history_ball_locations = []
isDraw = True
    
while True :
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    img_mask = cv2.inRange(img_hsv, (100, 200, 0), (140, 255, 255))
    
    # 모폴로지 연산 함수를 이용한 닫힘 연산(팽창 -> 침식)으로 틈 채우기
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (11,11))
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_CLOSE, kernel, iterations = 3)
    
    # 물체에 대한 경계 박스 그리기
    # cv2.connectedComponentsWithStats - 객체 정보를 함께 반환하는 레이블링 함수 (객체의 크기, 중심 위치 반환)
    retval, label, stats, centroids = cv2.connectedComponentsWithStats(img_mask)
    
    # 전체 라벨링 데이터에서 영역을 계산한 후 최대 영역 찾기
    max = -1
    max_index = -1
    for i in range(retval) : 
        if i < 1:
            continue
            
        area = stats[i, cv2.CC_STAT_AREA]
        
        if area > max :
            max = area
            max_index = i
            
        if max_index != -1 :
            
            center_x = int(centroids[max_index, 0])
            center_y = int(centroids[max_index, 1])
            left = stats[max_index, cv2.CC_STAT_LEFT]
            top = stats[max_index, cv2.CC_STAT_TOP]
            width = stats[max_index, cv2.CC_STAT_WIDTH]
            height = stats[max_index, cv2.CC_STAT_HEIGHT]
            
            # 객체의 경계 박스 그리기 및 객체의 중심에 점 그리기
            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 0, 255), 3)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
            
            # 그림 그리기 or 멈추기 설정
            if isDraw :
                list_ball_location.append((center_x, center_y))
            else :
                history_ball_locations.append(list_ball_location.copy())
                list_ball_location.clear()
            
    frame = draw_ball_location(frame, list_ball_location)
    for ball_locations in history_ball_locations :
        frame = draw_ball_location(frame, ball_locations)
            
    
    
    if not ret :
        print('frame read failed')
        break
    
#     cv2.imshow('Blue', img_mask)
    cv2.imshow('Result', frame)
    cv2.imshow('image', img)
    
    key = cv2.waitKey(20)
    if key == 27 :
        break
    elif key == 32 : # 스페이스바
        list_ball_location.clear()
        history_ball_locations.clear()
    elif key == ord('m') :
        isDraw = not isDraw

cap.release()
cv2.destroyAllWindows()

## webcam의 객체 tracking을 새로운 영상에 그리기

In [6]:
def draw_ball_location(img, locations) :
    for i in range(len(locations)-1) :
        if locations[0] is None or locations[1] is None :
            continue
        cv2.line(img1, tuple(locations[i]), tuple(locations[i+1]), (255, 0, 0), 7)  
    return img1

img1 = np.ones((500,700,3), np.uint8)*255

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
list_ball_location = []
history_ball_locations = []
isDraw = True
    
while True :
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     img_mask = cv2.inRange(img_hsv, (22, 120, 0), (42, 255, 255)) # 노란색 객체 인식
    img_mask = cv2.inRange(img_hsv, (100, 200, 0), (140, 255, 255)) # 파란색 객체 인식
    
    # 모폴로지 연산 함수를 이용한 열림 연산(침식 -> 팽창)으로 작은 픽셀 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_CLOSE, kernel, iterations = 3)
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_OPEN, kernel, iterations = 3)
    
    # 물체에 대한 경계 박스 그리기
    # cv2.connectedComponentsWithStats - 객체 정보를 함께 반환하는 레이블링 함수 (객체의 크기, 중심 위치 반환)
    retval, label, stats, centroids = cv2.connectedComponentsWithStats(img_mask)
    
    # 전체 라벨링 데이터에서 영역을 계산한 후 최대 영역 찾기
    max = -1
    max_index = -1
    for i in range(retval) : 
        if i < 1:
            continue
            
        area = stats[i, cv2.CC_STAT_AREA]
        
        if area > max :
            max = area
            max_index = i
            
        if max_index != -1 :
            
            center_x = int(centroids[max_index, 0])
            center_y = int(centroids[max_index, 1])
            left = stats[max_index, cv2.CC_STAT_LEFT]
            top = stats[max_index, cv2.CC_STAT_TOP]
            width = stats[max_index, cv2.CC_STAT_WIDTH]
            height = stats[max_index, cv2.CC_STAT_HEIGHT]
            
            # 객체의 경계 박스 그리기 및 객체의 중심에 점 그리기
            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 0, 255), 3)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
            
            # 그림 그리기 or 멈추기 설정
            if isDraw :
                list_ball_location.append((center_x, center_y))
            else :
                history_ball_locations.append(list_ball_location.copy())
                list_ball_location.clear()
            
    img1 = draw_ball_location(img1, list_ball_location)
    for ball_locations in history_ball_locations :
        img1 = draw_ball_location(img1, ball_locations)
    
    if not ret :
        print('frame read failed')
        break
    
#     cv2.imshow('Blue', img_mask)
    cv2.imshow('Result', frame)
    cv2.imshow('image 1', img1) 
    
    key = cv2.waitKey(20)
    
    if key == 27 :
        break
    elif key == 32 : # 스페이스바
        list_ball_location.clear()
        history_ball_locations.clear()
        img1 = np.ones((500,700,3), np.uint8)*255
    elif key == ord('m') :
        isDraw = not isDraw

cap.release()
cv2.destroyAllWindows()

## ( + 추가) 마우스 포인터 넣기

In [19]:
def draw_ball_location(img, locations) :
    img2 = img.copy()
    for i in range(len(locations)-1) :
        img1 = np.ones((500,700,3), np.uint8)*255
        if locations[0] is None or locations[1] is None :
            continue
        cv2.line(img, tuple(locations[i]), tuple(locations[i+1]), (255, 0, 0), 5)  
        cv2.circle(img1, tuple(locations[i+1]), 5, (0, 0, 255), -1)
        img2 = cv2.addWeighted(img, 0.7, img1, 0.3, 0)
        
    return img2
    

img = np.ones((500,700,3), np.uint8)*255
img1 = img.copy()

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
list_ball_location = []
history_ball_locations = []
isDraw = True

while True :
    ret, frame = cap.read()
    
    frame = cv2.flip(frame, 1)
    
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     img_mask = cv2.inRange(img_hsv, (22, 120, 0), (42, 255, 255)) # 노란색 객체 인식
    img_mask = cv2.inRange(img_hsv, (100, 200, 0), (140, 255, 255)) # 파란색 객체 인식
    
    # 모폴로지 연산 함수를 이용한 열림 연산(침식 -> 팽창)으로 작은 픽셀 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_CLOSE, kernel, iterations = 3)
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_OPEN, kernel, iterations = 3)
    
    # 물체에 대한 경계 박스 그리기
    # cv2.connectedComponentsWithStats - 객체 정보를 함께 반환하는 레이블링 함수 (객체의 크기, 중심 위치 반환)
    retval, label, stats, centroids = cv2.connectedComponentsWithStats(img_mask)
    
    # 전체 라벨링 데이터에서 영역을 계산한 후 최대 영역 찾기
    max = -1
    max_index = -1
    for i in range(retval) : 
        if i < 1:
            continue
            
        area = stats[i, cv2.CC_STAT_AREA]
        
        if area > max :
            max = area
            max_index = i
            
        if max_index != -1 :
            
            center_x = int(centroids[max_index, 0])
            center_y = int(centroids[max_index, 1])
            left = stats[max_index, cv2.CC_STAT_LEFT]
            top = stats[max_index, cv2.CC_STAT_TOP]
            width = stats[max_index, cv2.CC_STAT_WIDTH]
            height = stats[max_index, cv2.CC_STAT_HEIGHT]
            
            # 객체의 경계 박스 그리기 및 객체의 중심에 점 그리기
            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 0, 255), 3)
            cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)
            
            # 그림 그리기 or 멈추기 설정
            if isDraw :
                list_ball_location.append((center_x, center_y))
            else :
                history_ball_locations.append(list_ball_location.copy())
                list_ball_location.clear()
            
    img = draw_ball_location(img, list_ball_location)
    for ball_locations in history_ball_locations :
        img = draw_ball_location(img, ball_locations)    
    
    if not ret :
        print('frame read failed')
        break
    
    cv2.imshow('Blue', img_mask)
    cv2.imshow('Result', frame)
#     cv2.imshow('image 1', img1)
    cv2.imshow('image', img)
    
    
    key = cv2.waitKey(20)
    if key == 27 :
        break
    elif key == 32 : # 스페이스바
        list_ball_location.clear()
        history_ball_locations.clear()
    elif key == ord('m') :
        isDraw = not isDraw

cap.release()
cv2.destroyAllWindows()